# Calculate TCGI

Input Predictor: (ref. Camargo et al, 2014)
- Vorticity: clipped at $3.7 * 10^{-5}$ absolute vorticity @ 850-hPa [$1*10^{-5} s^{-1}$]
- Humidity: column-relative humidity [%]
- Thermal: potential intensity [m/s]
- Shear: vertical shear between the 850- and 200-hPa [m/s]

Coefficients:
- b (constant)
- bv, bh, bt, bs (corresponding to four predictors above)

In [1]:
import xarray as xr
import numpy as np
import xesmf as xe
import glob
import os
import re
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import warnings

In [4]:
models = ['MPI-ESM1-2-XR', 'CNRM-CM6-1-HR', 
          'HadGEM3-GC31-HM', 'HadGEM3-GC31-HM', 'HadGEM3-GC31-HM', 
          'EC-Earth3P-HR', 'EC-Earth3P-HR', 'EC-Earth3P-HR', 'EC-Earth3P-HR']
ens = ['r1i1p1f1', 'r1i1p1f2', 
       'r1i1p1f1', 'r1i2p1f1', 'r1i3p1f1',  
       'r1i1p2f1', 'r2i1p2f1', 'r1i1p1f1', 'r2i1p1f1', ]
experiments = ['highres_hm', 'highresSST_hm', 'highres_sm', 'highresSST_sm']
experiments_clim = ['highres', 'highresSST']
input = ['avort', 'swv', 'prw', 'PI', 'ws']
output = ['avort', 'crh', 'sd', 'PI', 'ws']

# Define the output for regridder with the horizontal resolution of 2*2deg
resolution = 2 
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 90+resolution, resolution)), 
                     'lon': (['lon'], np.arange(0, 360, resolution))})

### 1. Regrid varibales to 2*2 deg, Apply land mask

Original varibales and corresponding calculation codes
- avort (./calculation/calculate_avort.py)
- ws (./calculation/calculate_ws.py)
- swv (./calculation/calculate_swv.py)
- prw (direct output from models)
- PI (./calculation/tcpyPI/run_PI_highresmip)
- data storage path: /data0/yxia/TCGI/

A little calculation
- avort = abs(avort * 1e+05).clip(min=None, max=3.7)
- crh = prw / swv * 100
- sd = prw - swv

For PI & swv & prw, to avoid strange values in the coastal area after regrid
- Using the model's original mask to identify the land area.
- Fill the land area with zonal mean climatology PI at each latitude.

Saved files (path: /data0/yxia/TCGI/2deg)
- avort, ws, PI, crh, sd

Note: Land mask has a resolution of 2*2 deg (Source: WorldBath)

In [3]:
def replace_land(file_path, imodel, input_var):
    input_var = "vmax" if input_var == "PI" else input_var
    dataset = xr.open_dataset(file_path)
    
    # Input model's original land mask
    sftlf = xr.open_dataset(f"/data0/yxia/TCGI/mask/sftlf_{imodel}.nc").sftlf
    model_mask = sftlf.where(sftlf==0)
    
    # Get the climatology for every latitudes
    dataset_masked = dataset[input_var].where((model_mask == 0).data)
    ds_clim = dataset_masked.mean('time').mean(dim='lon', skipna=True)
    lat_mean = ds_clim.expand_dims(dim={'lon': dataset['lon'].data}, axis=-1)
    
    # Replace the land area with 'lat_mean'
    ds_smoothed = dataset[input_var].where((model_mask == 0).data, other=lat_mean)
    # drop coordinate 'type' if it exists
    ds_smoothed = ds_smoothed.drop_vars('type') if 'type' in ds_smoothed.coords else ds_smoothed
    return ds_smoothed

In [4]:
# input the land mask
land_mask = xr.open_dataset("/data0/yxia/TCGI/mask/LandMask_2deg.nc").lsm

In [6]:
for iexp in experiments:
    for imodel, iens in zip(models[2:5], ens[2:5]):
        ds = {}
        if not os.path.exists(f"/data0/yxia/TCGI/{imodel}/{iens}/avort/avort_{imodel}_{iens}_{iexp}.nc"):
            # If the file does not exist, skip this iteration of the loop
            print(f'### NO data for {iexp}_{imodel}_{iens}\n')
            continue

        print(f'### START 2deg {iexp}_{imodel}_{iens}')
        for input_var in input:
            # Input data listed above
            file_path = f"/data0/yxia/TCGI/{imodel}/{iens}/{input_var}/{input_var}_{imodel}_{iens}_{iexp}.nc"
            if input_var in ['swv', 'prw', 'PI']:
                ds_raw = replace_land(file_path, imodel, input_var)
            else:
                ds_raw = xr.open_dataset(file_path)[input_var]
            # Regrid and store in dsr
            regridder = xe.Regridder(ds_raw, ds_out, 'bilinear', periodic=True)
            dsr = regridder(ds_raw)
            # mask the land
            ds[f"{input_var}"] = dsr.where(~np.isnan(land_mask), other=np.nan)

        # del ds_raw, dsr, regridder

        # Calculate variables and change units 
        avort = abs(ds['avort'] * 1e+5).clip(min=None, max=3.7)   # [10-5 * s-1]
        crh = ds['prw'] / ds['swv'] * 100     # [%]
        sd = ds['prw'] - ds['swv']            # [g m-2]
        PI = ds['PI']   # [m s-1]
        ws = ds['ws']   # [m s-1]
        del ds
        
        # Assign attributes to each variable
        attributes = {
            'avort': {'units': '10-5 * s-1', 
                      'calculation': 'take the magitude of absolute vorticity and clip at 3.7'},
            'crh': {'units': '%', 
                    'calculation': 'atmosphere_water_vapor / saturated_water_vapor * 100'},
            'sd': {'units': 'kg m-2', 
                   'calculation': 'atmosphere_water_vapor - saturated_water_vapor'},
            'PI': {'units': 'm s-1', 
                   'calculation': 'PI_vmax'},
            'ws': {'units': 'm s-1', 
                   'calculation': 'wind_shear'}
        }

        # Save regridded & masked files to /data0/yxia/TCGI/2deg
        for var in output:
            eval(var).attrs['units'] = attributes[var]['units']
            eval(var).attrs['calculation'] = attributes[var]['calculation']
            eval(var).rename(var).to_netcdf(f"/data0/yxia/TCGI/2deg/{imodel}/{iens}/{var}/{var}_{imodel}_{iens}_{iexp}_2deg.nc", \
                                            encoding={f'{var}': {'zlib': True, 'complevel': 5}}, mode="w")
            print(f'SAVED 2deg {imodel}_{iens}_{var}')
        print('')
        del avort, crh, sd, PI, ws

### START 2deg highres_hm_HadGEM3-GC31-HM_r1i1p1f1
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_avort
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_crh
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_sd
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_PI
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_ws

### START 2deg highres_hm_HadGEM3-GC31-HM_r1i2p1f1
SAVED 2deg HadGEM3-GC31-HM_r1i2p1f1_avort
SAVED 2deg HadGEM3-GC31-HM_r1i2p1f1_crh
SAVED 2deg HadGEM3-GC31-HM_r1i2p1f1_sd
SAVED 2deg HadGEM3-GC31-HM_r1i2p1f1_PI
SAVED 2deg HadGEM3-GC31-HM_r1i2p1f1_ws

### START 2deg highres_hm_HadGEM3-GC31-HM_r1i3p1f1
SAVED 2deg HadGEM3-GC31-HM_r1i3p1f1_avort
SAVED 2deg HadGEM3-GC31-HM_r1i3p1f1_crh
SAVED 2deg HadGEM3-GC31-HM_r1i3p1f1_sd
SAVED 2deg HadGEM3-GC31-HM_r1i3p1f1_PI
SAVED 2deg HadGEM3-GC31-HM_r1i3p1f1_ws

### START 2deg highresSST_hm_HadGEM3-GC31-HM_r1i1p1f1
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_avort
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_crh
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_sd
SAVED 2deg HadGEM3-GC31-HM_r1i1p1f1_PI
SAVED 2deg HadGEM3-GC31-HM_r1i1p1

### 2. Calculate climatology of year 1981-2010 (12 months)
Note: if one model has multiple ensembles, the climatology is multi-ensemble mean.

In [7]:
def find_files(path, name_pattern):
    # Create a pattern that matches the files in the provided path and all subdirectories
    full_pattern = os.path.join(path, '**', name_pattern)

    # Use glob.glob with recursive=True to search in subdirectories
    files = glob.glob(full_pattern, recursive=True)
    return files

def get_ens_clim(files):
    # function to get the ensemble mean climatology
    monthly_clim = []
    for file in files:
        ds = xr.open_dataset(file)
        monthly_mean = ds.sel(time=slice('1981', '2010')).groupby('time.month').mean()
        monthly_clim.append(monthly_mean)
    # Combine all the monthly means and calculate the average across all files
    ds_clim = sum(monthly_clim) / len(monthly_clim)
    return ds_clim

In [13]:
for iexp_clim in experiments_clim:
    for imodel, iens in zip(models[2:3], ens[2:3]):    # only need one group of input for one model
        print(f'### START clim {iexp_clim}-{imodel}-{iens}')
        for var in output:
            if imodel in ['MPI-ESM1-2-XR', 'CNRM-CM6-1-HR']:
                ds = xr.open_dataset(f"/data0/yxia/TCGI/2deg/{imodel}/{iens}/{var}/{var}_{imodel}_{iens}_{iexp_clim}_hm_2deg.nc")
                ds_clim = ds.sel(time=slice('1981', '2010')).groupby('time.month').mean()
            else:
                path = f'/data0/yxia/TCGI/2deg/{imodel}/'
                name_pattern = f'{var}_{imodel}_*_{iexp_clim}_hm_2deg.nc'
                my_files = find_files(path, name_pattern)
                ds_clim = get_ens_clim(my_files)
            ds_clim[var].attrs['Notes'] = 'Monthly climatology (year 1981-2010)'
            ds_clim.to_netcdf(f"/data0/yxia/TCGI/clim/clim_highres/{imodel}/{var}/{var}_{imodel}_{iexp_clim}_clim_1981_2010_2deg.nc", \
                              encoding={f'{var}': {'zlib': True, 'complevel': 5}}, mode="w")
            print(f'SAVED clim {imodel}_{var}')
        print('')
del ds_clim

### START clim highres-HadGEM3-GC31-HM-r1i1p1f1
SAVED clim HadGEM3-GC31-HM_avort
SAVED clim HadGEM3-GC31-HM_crh
SAVED clim HadGEM3-GC31-HM_sd
SAVED clim HadGEM3-GC31-HM_PI
SAVED clim HadGEM3-GC31-HM_ws

### START clim highresSST-HadGEM3-GC31-HM-r1i1p1f1
SAVED clim HadGEM3-GC31-HM_avort
SAVED clim HadGEM3-GC31-HM_crh
SAVED clim HadGEM3-GC31-HM_sd
SAVED clim HadGEM3-GC31-HM_PI
SAVED clim HadGEM3-GC31-HM_ws



### If files (2deg & clim) are ready, run the code from here

In [2]:
def add_climatology(month_group, climatology):
    # Adjusted function to add climatology
    
    # Extract the month number from the group
    month_num = month_group.time.dt.month.values[0]
    
    # Select the climatology data for the corresponding month
    climatology_month = climatology.sel(month=month_num)
    
    # Add the climatology to the month_group
    # The 'align' function ensures that the lat and lon dimensions match
    month_group, climatology_month = xr.align(month_group, climatology_month)
    return month_group + climatology_month

In [5]:
# Coefficients sequence:
# [constant, vorticity, humidity, thermal, shear]
b_crh = [-24.1323, 2.5120, 0.0770, 0.0622, -0.1202]   # CRH & PI
b_sd = [-18.3563, 2.4829, 0.0735, 0.0798, -0.1346]   # SD & PI

area = np.log(np.cos(np.deg2rad(ds_out.lat))*2*2)
area_ext = np.tile(area, (101*12, len(ds_out.lon), 1)).transpose(0,2,1)  # Rearrange dimensions to (time, lat, lon)

clim_era5 = {}
for var in output:
    clim_era5[f"{var}"] = xr.open_dataset(f"/data0/yxia/TCGI/clim/clim_ERA5/{var}_ERA5_clim_1981_2010_2deg.nc")

In [19]:
for imodel, iens in zip(models[1:], ens[1:]): # Only select the first two ensembles for EC model
    '''
    3. Subtract ERA5 climatology from calculated highresmip climatology
    (Because we'll use coeffient b from ERA5 later.
     Step 3 & 4 are for fixing the bias between the model and ERA5)
    '''
    clim_high = {}; clim_highSST = {}; cor_highres = {}; cor_highresSST = {};
    for var in output:
        clim_high[f"{var}"] = xr.open_dataset(f"/data0/yxia/TCGI/clim/clim_highres/{imodel}/{var}/{var}_{imodel}_highres_clim_1981_2010_2deg.nc")
        clim_highSST[f"{var}"] = xr.open_dataset(f"/data0/yxia/TCGI/clim/clim_highres/{imodel}/{var}/{var}_{imodel}_highresSST_clim_1981_2010_2deg.nc")
        cor_highres[f"{var}"] = clim_era5[f"{var}"][var] - clim_high[f"{var}"][var]
        cor_highresSST[f"{var}"] = clim_era5[f"{var}"][var] - clim_highSST[f"{var}"][var]
    print(f'GET correction for {imodel}')

    '''
    4. Add correction term to highres data
    '''
    bc_highres_hm = {}; bc_highresSST_hm = {}; bc_highres_sm = {}; bc_highresSST_sm = {}
    bc_ds = [bc_highres_hm, bc_highresSST_hm, bc_highres_sm, bc_highresSST_sm]
    
    for iexp, ibc_ds in zip(experiments, bc_ds):
        if imodel == 'EC-Earth3P-HR' and iexp in ['highresSST_hm', 'highresSST_sm']:
            iens = re.sub(r'p\d', 'p1', iens) # change the number after 'p' to '1'
        elif imodel == 'EC-Earth3P-HR' and iexp in ['highres_hm', 'highres_sm']:
            iens = re.sub(r'p\d', 'p2', iens)
        for var in output:
            # Apply the add_climatology function to each month
            ori_ds = xr.open_dataset(f"/data0/yxia/TCGI/2deg/{imodel}/{iens}/{var}/{var}_{imodel}_{iens}_{iexp}_2deg.nc")[var]
            correction = cor_highres if iexp in ['highres_hm', 'highres_sm'] else cor_highresSST
            ibc_ds[f"{var}"] = ori_ds.groupby('time.month').apply(add_climatology, climatology=correction[f"{var}"])
    print(f'Correction added for {imodel}-{iens}')

    # Combine historial and future simulations along time series
    # store them into bc_full
    bc_highres = {}; bc_highresSST = {}
    bc_full = [bc_highres, bc_highresSST]

    for iexp_clim in range(2):   # loop for highres & highresSST
        for var in output:
            if 'lev' in bc_highres_hm[var].coords:
                hm = bc_ds[iexp_clim][var].drop_vars(['month', 'lev'])
                sm = bc_ds[iexp_clim+2][var].drop_vars(['month', 'lev'])
            elif 'plev' in bc_highres_hm[var].coords:
                hm = bc_ds[iexp_clim][var].drop_vars(['month', 'plev'])
                sm = bc_ds[iexp_clim+2][var].drop_vars(['month', 'plev'])
            else:
                hm = bc_ds[iexp_clim][var].drop_vars(['month'])
                sm = bc_ds[iexp_clim+2][var].drop_vars(['month'])
            combined_ds = xr.concat([hm, sm], dim='time')
            bc_full[iexp_clim][f'{var}'] = combined_ds
    del bc_ds

    '''
    5. Calculate TCGI
    using coeffients get from ERA5 (2deg)
    '''
    offset = xr.DataArray(area_ext, dims=['time', 'lat', 'lon'], 
                          coords={'time':bc_full[0]['avort']['time'], 'lon': ds_out.lon, 'lat': ds_out.lat})

    for iexp_clim in experiments_clim:   # loop for highres & highresSST
        bc_now = eval('bc_'+iexp_clim)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            mu_crh = np.exp(b_crh[0] + b_crh[1]*bc_now['avort'] + b_crh[2]*bc_now['crh'] + 
                            b_crh[3]*bc_now['PI'] + b_crh[4]*bc_now['ws'] + offset)
            mu_sd = np.exp(b_sd[0] + b_sd[1]*bc_now['avort'] + b_sd[2]*bc_now['sd'] + 
                           b_sd[3]*bc_now['PI'] + b_sd[4]*bc_now['ws'] + offset)
        if imodel in ['MPI-ESM1-2-XR', 'CNRM-CM6-1-HR']:
            for var in output:
                bc_now[var].to_netcdf(f"/data0/yxia/TCGI/unbiased/{imodel}/{var}_{imodel}_{iexp_clim}.nc", \
                                      encoding={f'{var}': {'zlib': True, 'complevel': 5}}, mode="w")
            # mu_crh.rename('TCGI').to_netcdf(f"/data0/yxia/TCGI/TCGI/{imodel}/TCGI-CRH_{imodel}_{iexp_clim}.nc", \
            #                                 encoding={'TCGI': {'zlib': True, 'complevel': 5}}, mode="w")
            # mu_sd.rename('TCGI').to_netcdf(f"/data0/yxia/TCGI/TCGI/{imodel}/TCGI-SD_{imodel}_{iexp_clim}.nc", \
            #                                 encoding={'TCGI': {'zlib': True, 'complevel': 5}}, mode="w")
        else:
            for var in output:
                bc_now[var].to_netcdf(f"/data0/yxia/TCGI/unbiased/{imodel}/{var}_{imodel}_{iens}_{iexp_clim}.nc", \
                                      encoding={f'{var}': {'zlib': True, 'complevel': 5}}, mode="w")
            # mu_crh.rename('TCGI').to_netcdf(f"/data0/yxia/TCGI/TCGI/{imodel}/TCGI-CRH_{imodel}_{iens}_{iexp_clim}.nc", \
            #                                 encoding={'TCGI': {'zlib': True, 'complevel': 5}}, mode="w")
            # mu_sd.rename('TCGI').to_netcdf(f"/data0/yxia/TCGI/TCGI/{imodel}/TCGI-SD_{imodel}_{iens}_{iexp_clim}.nc", \
            #                                 encoding={'TCGI': {'zlib': True, 'complevel': 5}}, mode="w")
        print(f"SAVED TCGI {iexp_clim} {imodel}_{iens}")
    print(" ")

print("### DONE")

GET correction for CNRM-CM6-1-HR
Correction added for CNRM-CM6-1-HR-r1i1p1f2
SAVED TCGI highres CNRM-CM6-1-HR_r1i1p1f2
SAVED TCGI highresSST CNRM-CM6-1-HR_r1i1p1f2
 
GET correction for HadGEM3-GC31-HM
Correction added for HadGEM3-GC31-HM-r1i1p1f1
SAVED TCGI highres HadGEM3-GC31-HM_r1i1p1f1
SAVED TCGI highresSST HadGEM3-GC31-HM_r1i1p1f1
 
GET correction for HadGEM3-GC31-HM
Correction added for HadGEM3-GC31-HM-r1i2p1f1
SAVED TCGI highres HadGEM3-GC31-HM_r1i2p1f1
SAVED TCGI highresSST HadGEM3-GC31-HM_r1i2p1f1
 
GET correction for HadGEM3-GC31-HM
Correction added for EC-Earth3P-HR-r1i1p1f1
SAVED TCGI highres EC-Earth3P-HR_r1i1p1f1
SAVED TCGI highresSST EC-Earth3P-HR_r1i1p1f1
 
GET correction for EC-Earth3P-HR
Correction added for EC-Earth3P-HR-r2i1p1f1
SAVED TCGI highres EC-Earth3P-HR_r2i1p1f1
SAVED TCGI highresSST EC-Earth3P-HR_r2i1p1f1
 
GET correction for EC-Earth3P-HR
Correction added for EC-Earth3P-HR-r1i1p1f1
SAVED TCGI highres EC-Earth3P-HR_r1i1p1f1
SAVED TCGI highresSST EC-Earth3P-

In [ ]:
for imodel in ['HadGEM3-GC31-HM', 'EC-Earth3P-HR']:
    for iexp_clim in experiments_clim:
        for var in output:
            path = f'/data0/yxia/TCGI/unbiased/{imodel}/'
            name_pattern = f'{var}_{imodel}_*_{iexp_clim}.nc'
            my_files = find_files(path, name_pattern)
            ens_mean = get_ens_mean(my_files)
            ens_mean.to_netcdf(f"/data0/yxia/TCGI/unbiased/{imodel}/{var}_{imodel}_{iexp_clim}.nc", 
                               encoding={f'{var}': {'zlib': True, 'complevel': 5}}, mode="w")
            print(f"SAVED {var} Ensemble mean for {imodel}_{iexp_clim}")

SAVED avort Ensemble mean for HadGEM3-GC31-HM_highres
SAVED crh Ensemble mean for HadGEM3-GC31-HM_highres
SAVED sd Ensemble mean for HadGEM3-GC31-HM_highres
SAVED PI Ensemble mean for HadGEM3-GC31-HM_highres
SAVED ws Ensemble mean for HadGEM3-GC31-HM_highres
SAVED avort Ensemble mean for HadGEM3-GC31-HM_highresSST
SAVED crh Ensemble mean for HadGEM3-GC31-HM_highresSST
SAVED sd Ensemble mean for HadGEM3-GC31-HM_highresSST
SAVED PI Ensemble mean for HadGEM3-GC31-HM_highresSST
SAVED ws Ensemble mean for HadGEM3-GC31-HM_highresSST
SAVED avort Ensemble mean for EC-Earth3P-HR_highres
SAVED crh Ensemble mean for EC-Earth3P-HR_highres
SAVED sd Ensemble mean for EC-Earth3P-HR_highres
SAVED PI Ensemble mean for EC-Earth3P-HR_highres
SAVED ws Ensemble mean for EC-Earth3P-HR_highres
SAVED avort Ensemble mean for EC-Earth3P-HR_highresSST
SAVED crh Ensemble mean for EC-Earth3P-HR_highresSST
SAVED sd Ensemble mean for EC-Earth3P-HR_highresSST


In [16]:
for imodel in ['HadGEM3-GC31-HM', 'EC-Earth3P-HR']:
    for iexp_clim in experiments_clim:
        for itype in ['CRH', 'SD']:
            path = f'/data0/yxia/TCGI/TCGI/{imodel}/'
            name_pattern = f'TCGI-{itype}_{imodel}_*_{iexp_clim}.nc'
            my_files = find_files(path, name_pattern)
            ens_mean = get_ens_mean(my_files)
            ens_mean.to_netcdf(f"/data0/yxia/TCGI/TCGI/{imodel}/TCGI-{itype}_{imodel}_{iexp_clim}.nc", 
                               encoding={'TCGI': {'zlib': True, 'complevel': 5}}, mode="w")
            print(f"SAVED TCGI Ensemble mean for {imodel}_{iexp_clim}")

SAVED TCGI Ensemble mean for HadGEM3-GC31-HM_highres
SAVED TCGI Ensemble mean for HadGEM3-GC31-HM_highres
SAVED TCGI Ensemble mean for HadGEM3-GC31-HM_highresSST
SAVED TCGI Ensemble mean for HadGEM3-GC31-HM_highresSST
SAVED TCGI Ensemble mean for EC-Earth3P-HR_highres
SAVED TCGI Ensemble mean for EC-Earth3P-HR_highres
SAVED TCGI Ensemble mean for EC-Earth3P-HR_highresSST
SAVED TCGI Ensemble mean for EC-Earth3P-HR_highresSST
